In [1]:
from caveclient import CAVEclient
from cloudvolume import CloudVolume
from meshparty import trimesh_vtk
import trimesh
import io
import trimesh
import numpy as np
import tifffile as tiff
from scipy import ndimage


In [2]:
cv = CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300", use_https=True)

mesh = cv.mesh.get(864691136194411734, lod=2)[864691136194411734]
mesh_actor = trimesh_vtk.mesh_actor(mesh,
                                    color=(1,0,0),
                                    opacity=0.5)
mesh_obj=(mesh.to_obj())
type(mesh_obj)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.02it/s]


bytes

In [3]:
print(mesh.vertices.shape, mesh.faces.shape)

(31237, 3) (58510, 3)


In [13]:
obj_bytes = mesh.to_obj()
mesh_trimesh = trimesh.load(io.BytesIO(obj_bytes), file_type='obj')
type(mesh_trimesh)

trimesh.base.Trimesh

In [15]:
mesh_trimesh.show()

In [21]:
def mesh_to_voxel_mask(mesh, voxel_resolution=128):
    """Convert trimesh to binary 3D voxel mask"""
    
    voxels = mesh.voxelized(pitch=mesh.extents.max() / voxel_resolution)
    voxels = voxels.fill()
    mask = voxels.matrix
    
    return mask

# Usage
binary_mask = mesh_to_voxel_mask(mesh_trimesh, voxel_resolution=1024)

In [23]:
mask_int = binary_mask.astype(np.uint8) * 255  # or np.int32
tiff.imwrite("/Users/kashika/Desktop/Haynes_Lab/filled_mesh_volume_2.tif", mask_int)

# some basic info
print(f"Shape: {mask_int.shape}")
print(f"Number of voxels inside mesh: {mask_int.sum()}")
print(f"Percentage filled: {mask_int.sum() / mask_int.size * 100:.2f}%")

Shape: (1025, 940, 669)
Number of voxels inside mesh: 299272335
Percentage filled: 46.43%


In [ ]:
# 1. Pad so we can identify the "outside" background easily
padded = np.pad(binary_mask.astype(bool), pad_width=1, mode='constant', constant_values=False)

# 2. Flood-fill from outside (background connected to borders)
empty = ~padded
labels, num = ndimage.label(empty, structure=np.ones((3,3,3), dtype=int))

# Identify which label touches the border → that’s the outside
border_labels = np.unique(np.concatenate([
    labels[0, :, :].ravel(),
    labels[-1, :, :].ravel(),
    labels[:, 0, :].ravel(),
    labels[:, -1, :].ravel(),
    labels[:, :, 0].ravel(),
    labels[:, :, -1].ravel()
]))

outside_mask = np.isin(labels, border_labels)

# 3. Everything NOT outside is inside (the filled object)
filled_padded = ~outside_mask
filled = filled_padded[1:-1, 1:-1, 1:-1]  # remove padding

# 4. Convert to uint8 (255 for object)
filled_u8 = filled.astype(np.uint8) * 255

print("Original surface voxels:", binary_mask.sum())
print("Filled voxels:", filled.sum())

# 5. Save as a 3D TIFF stack
tiff.imwrite('binary_mask_filled.tif', filled_u8, photometric='minisblack')

print("Saved binary_mask_filled.tif ✅")

In [ ]:
print(f"Volume: {mesh_trimesh.volume:.2f} cubic units")
print(f"Surface Area: {mesh_trimesh.area:.2f} square units")

In [ ]:
# Bounding box 
bounds = mesh_trimesh.bounds  # [[min_x, min_y, min_z], [max_x, max_y, max_z]]
extents = mesh_trimesh.extents  # [width, height, depth]
print(f"Dimensions (W×H×D): {extents[0]:.2f} × {extents[1]:.2f} × {extents[2]:.2f}")

In [ ]:
# Center of mass
print(f"Center of mass: {mesh_trimesh.center_mass}")
print(f"Centroid: {mesh_trimesh.centroid}")